# Multivariate Linear Regression

We are using airfoil and yatch hydrodynamics datasets to build a multivariate linear regression model.

In [5]:
# Import all required packages
import pandas as pd
import numpy as np
import random as random
from numpy.linalg import inv
import matplotlib.pyplot as py

#import air foil data and rename the column headers
airFoilData = pd.read_table("airfoil_self_noise.dat", sep="\s+",header=None)
airFoilData.columns = ['Freq','angle','chordLength','velocity','displacementThickness','soundLevel']

#import yatch hydrodynamics data and rename the column headers
yatchData = pd.read_table("yacht_hydrodynamics.data", sep="\s+",header=None)
yatchData.columns = ['buoy','prismatic','lenDispRatio','beamDraftRatio','lenBeamRatio','froudeNum','residuaryRes']

#import concrete slump data and take the first row as column headers, also remove the row number column
slumpData = pd.read_table("slump_test.data", sep=",",header=0)
slumpData = slumpData.drop('No',axis=1)

In [6]:
# Define a function to perform 1) Linear multiple regression and 2) RBF regression
def my_regression(train,test,nOutputs,lamda): # takes inputs train, test,num of Outputs, lamda(regression co-efficient)
    # model 1  - Regularized Linear multiple regression      
    Y=train[:,-nOutputs:] # create a matrix of outputs of train data
    X = train[:,:-nOutputs]    # create a matrix of inputs of test data
    nFeatures = X.shape[1]  # number of features in dataset
    nRows = X.shape[0]    # number of rows in train data
    
    # calculate the weights or parameters using the training data in matrix notation
    beta = np.dot(np.dot(inv(lamda*np.identity(nFeatures)+np.dot(X.transpose(),X)),X.transpose()),Y)    
    
    Y_test = test[:,-nOutputs:] # create a matrix of outputs of test data 
    X_test = test[:,:-nOutputs] # create a matrix of inputs of test data
    nRows_test = X_test.shape[0]    
    
    Y_est = np.dot(X_test,beta) # estimate the Y-Hat for test data inputs using the weights we got from train data.
    # Calculate least suare error for given test data
    LSE = (((np.square(Y_test-Y_est)).sum()))+(lamda/2)*(np.dot(beta.transpose(),beta)) 
    
    
    # model 2 - RBF --- this part of the code performs non-linear regression using Radial basis functions
    sd = 1 # set standard deviation to 1
    phi = np.empty((nRows,10)) 
    phi_test = np.empty((nRows_test,10))
    
    # Prep the input train data to calculate PHI or the design matrix
    X_RBF = np.delete(X, 0, 1) 
    X_RBF_test = np.delete(X_test,0,1)
    
    # choose a random row from input as mean values -- here i am taking 10 so the feature space is expanded to 10 dimensions
    mean = X_RBF[np.random.randint(10, size=2),:]    
    cind=0
    rind = 0
    cind_test=0
    rind_test = 0
    
    # the below loops are used to calculate the design matrix using radial basis matrix.
    for m in mean:
        for row in range(nRows):
            dist = np.linalg.norm(X_RBF[row,:]-m)        # calculate euclidean distance between the points
            gauss = np.exp((-1*(dist**2))/(2*(sd**2)))  # calculate gaussian value
            phi[rind,cind] = gauss
            rind = rind + 1
        rind = 0 
        cind = cind+1
        
        for row_test in range(nRows_test):        
            dist_test = np.linalg.norm(X_RBF_test[row_test,:]-m)   # calculate euclidean distance between the points         
            gauss_test = np.exp((-1*(dist_test**2))/(2*(sd**2))) # calculate gaussian value        
            phi_test[rind_test,cind_test] = gauss_test
            rind_test = rind_test + 1
        rind_test = 0 
        cind_test = cind_test+1  
    np.insert(phi, 0, 1, axis=1)    
    np.insert(phi_test, 0, 1, axis=1)    
    
    # calculate parameters or weights for the given design matrix
    beta_RBF = np.dot(np.dot(inv(lamda*np.identity(10)+np.dot(phi.transpose(),phi)),phi.transpose()),Y)
    Y_est_RBF = np.dot(phi_test,beta_RBF) # estimate the Y-Hat for test data inputs using the weights we got from train data.
    
    #calculate least square error for gaussian regression
    LSE_RBF = (((np.square(Y_test-Y_est_RBF)).sum()))+(lamda/2)*(np.dot(beta_RBF.transpose(),beta_RBF))
    return beta,LSE,beta_RBF,LSE_RBF # return least sqares and weights for both the model

def process(df,lamda,outputs):
    k = 5 # choose number of folds
    padding = len(df) % k
    cols = df.columns
    LSE = []
    LSE_RBF = []
    beta = []
    beta_RBF = []
    train= pd.DataFrame()
    test = pd.DataFrame()
    
    # normalize the entire data using Z-scoring method
    for u in range(df.shape[1]):
            df[cols] = (df[cols]-df[cols].mean())/(df[cols].std())
    #insert a column of '1's for W0 parameter
    df.insert(0,'dummy',1)

    init_split = np.split(df[:-padding],k)
    init_split[len(init_split)-1].append(df.tail(padding))

    for h in range(k): # iterate through each fold
        for j in range(len(init_split)-1):
            train = train.append(init_split[j])
    
        train = train.as_matrix() #create the train data
        test = init_split[len(init_split)-1].as_matrix()    # create the test data 
        
        # call the my_regression function for to model a regression model and obtain weights and least square errors
        weight,LSE_mid,weight_RBF,LSE_mid_RBF = my_regression(train,test,outputs,lamda)
        
        # capture the weights and least squares of each fold
        beta.append(weight)
        LSE.append(LSE_mid)
        beta_RBF.append(weight_RBF)
        LSE_RBF.append(LSE_mid_RBF)
        
        train= pd.DataFrame()
        test = pd.DataFrame()
        init_split = init_split[(h+1):] + init_split[:(h+1)]
    return np.average(LSE) # return the average least square error for the given dataset

print("Least square error for 'Airfoil Self-Noise' data with 5 fold cross validation using Linear multiple regression")
print(process(airFoilData,65,1))
print("Least square error for 'Yacht Hydrodynamics' data with 5 fold cross validation using Linear multiple regression")
print(process(yatchData,2,1))
print("Least square error for 'Concrete Slump' data with 5 fold cross validation using Linear multiple regression")
print(process(slumpData,1,3))

Least square error for 'Airfoil Self-Noise' data with 5 fold cross validation using Linear multiple regression


C:\Users\sriha\Anaconda3\lib\site-packages\ipykernel_launcher.py:89: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\sriha\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


146.75958584021924
Least square error for 'Yacht Hydrodynamics' data with 5 fold cross validation using Linear multiple regression
21.190315297113585

C:\Users\sriha\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: overflow encountered in square



Least square error for 'Concrete Slump' data with 5 fold cross validation using Linear multiple regression
31.247879614569225
